In [13]:
from tensorflow import keras
from keras.preprocessing import image
import numpy as np
import pandas as pd
import os
from PIL import Image
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from keras.models import load_model
import tensorflow.keras as kr
import tensorflow.keras.layers as kl
from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping

In [14]:
#Declare Path
train_csv_path = "./Dataset/train/"
test_csv_path = "./Dataset/test/"
valid_csv_path = "./Dataset/valid/"

In [15]:
#Load
categories = ['Brazil', 'Canada', 'Finland', 'Japan', 'United_States', 'United-Kingdom']

In [16]:
#Pre process image 
def preprocess_image(image):
    image = image.resize((224, 224))

    if image.mode != 'RGB':
        image = image.convert('RGB')
    
    img_array = np.array(image)
    img_array = img_array / 255.0
    return img_array

In [17]:
#Pre process dataset
def preprocess_dataset():
    df = pd.read_csv(os.path.join(train_csv_path, "_classes.csv"))
    df.columns = df.columns.str.strip().str.lower()

    images = []
    labels = []

    error_count = 0

    for index, row in df.iterrows():
        try:    
            image_path = os.path.join(train_csv_path, row['filename'])
            image = Image.open(image_path)
            processed_image = preprocess_image(image)
            images.append(processed_image)

            label = [category for category in categories if row[category.lower()] == 1]
            labels.append(label)
        except Exception as e:
            print(f"Error {row['filename']} : {e}")
            error_count += 1
            
    print(error_count)
    return np.array(images), labels

    

In [18]:
#Fix shape after converting
def fix_shape(image, label):
    image = tf.reshape(image, (224, 224, 3))
    return image, label

In [19]:
images, labels = preprocess_dataset()

Error canvas_1629740587_jpg.rf.a41cf0ce826cc9c470034c880eeaab96.jpg : [Errno 2] No such file or directory: './Dataset/train/canvas_1629740587_jpg.rf.a41cf0ce826cc9c470034c880eeaab96.jpg'
Error canvas_1629737804_jpg.rf.a4f549f9dc0ef649a529b1fb3c6b7073.jpg : [Errno 2] No such file or directory: './Dataset/train/canvas_1629737804_jpg.rf.a4f549f9dc0ef649a529b1fb3c6b7073.jpg'
Error canvas_1629737601_jpg.rf.a5b690291920295d35e712a72f5e4dbb.jpg : [Errno 2] No such file or directory: './Dataset/train/canvas_1629737601_jpg.rf.a5b690291920295d35e712a72f5e4dbb.jpg'
Error canvas_1629738048_jpg.rf.a6f7eae363da510aa69bbc16866cd3f3.jpg : [Errno 2] No such file or directory: './Dataset/train/canvas_1629738048_jpg.rf.a6f7eae363da510aa69bbc16866cd3f3.jpg'
Error canvas_1629745936_jpg.rf.a713ab831dabe7b1ebf87f8d94b6546b.jpg : [Errno 2] No such file or directory: './Dataset/train/canvas_1629745936_jpg.rf.a713ab831dabe7b1ebf87f8d94b6546b.jpg'
Error canvas_1629739324_jpg.rf.a99076fa4737bc0522035980e3d0513d.j

In [20]:
flattened_labels = [label[0] for label in labels if label]
lookup = tf.keras.layers.StringLookup(output_mode='int')
lookup.adapt(flattened_labels)

integer_labels = lookup(flattened_labels)
one_hot_labels = tf.one_hot(integer_labels, depth=6)

#Filter image also
filtered_images = [image for image, label in zip(images, labels) if label]

In [21]:
#Check value nya label
for i, label_list in enumerate(one_hot_labels):
    print(f"Item {i+1}: {label_list}")

Item 1: [0. 0. 0. 0. 1. 0.]
Item 2: [0. 0. 0. 0. 0. 0.]
Item 3: [0. 1. 0. 0. 0. 0.]
Item 4: [0. 0. 1. 0. 0. 0.]
Item 5: [0. 0. 0. 0. 0. 1.]
Item 6: [0. 0. 0. 0. 0. 1.]
Item 7: [0. 1. 0. 0. 0. 0.]
Item 8: [0. 0. 0. 0. 0. 0.]
Item 9: [0. 0. 0. 0. 0. 0.]
Item 10: [0. 0. 0. 1. 0. 0.]
Item 11: [0. 0. 1. 0. 0. 0.]
Item 12: [0. 0. 0. 0. 1. 0.]
Item 13: [0. 0. 0. 0. 0. 1.]
Item 14: [0. 0. 0. 0. 0. 1.]
Item 15: [0. 0. 0. 1. 0. 0.]
Item 16: [0. 1. 0. 0. 0. 0.]
Item 17: [0. 0. 0. 1. 0. 0.]
Item 18: [0. 0. 1. 0. 0. 0.]
Item 19: [0. 0. 0. 0. 1. 0.]
Item 20: [0. 1. 0. 0. 0. 0.]
Item 21: [0. 0. 0. 0. 1. 0.]
Item 22: [0. 0. 1. 0. 0. 0.]
Item 23: [0. 1. 0. 0. 0. 0.]
Item 24: [0. 0. 1. 0. 0. 0.]
Item 25: [0. 0. 0. 0. 1. 0.]
Item 26: [0. 1. 0. 0. 0. 0.]
Item 27: [0. 0. 0. 0. 0. 1.]
Item 28: [0. 0. 0. 1. 0. 0.]
Item 29: [0. 0. 0. 0. 0. 0.]
Item 30: [0. 0. 1. 0. 0. 0.]
Item 31: [0. 0. 0. 1. 0. 0.]
Item 32: [0. 0. 1. 0. 0. 0.]
Item 33: [0. 0. 0. 0. 0. 0.]
Item 34: [0. 0. 0. 0. 0. 1.]
Item 35: [0. 0. 0. 1. 0

In [22]:
print(f"label {len(integer_labels)}, image {len(filtered_images)}")

label 6140, image 6140


In [23]:
print(f"label {type(labels)}, image {type(images)}")

label <class 'list'>, image <class 'numpy.ndarray'>


In [24]:
converted_train__data = tf.data.Dataset.from_tensor_slices((filtered_images, one_hot_labels)).batch(16)

In [25]:
for batch in converted_train__data.take(1):
    images_batch, labels_batch = batch
    print("Batch shape:", images_batch.shape)
    print("Single image shape:", images_batch[0].shape)
    print("Batch labels shape:", labels_batch.shape)

Batch shape: (16, 224, 224, 3)
Single image shape: (224, 224, 3)
Batch labels shape: (16, 6)


In [26]:
# converted_train_data = converted_train__data.map(fix_shape)

In [27]:
num_classes = len(categories)

In [28]:
def alexnet(num_of_classes):
    model = kr.Sequential([
        # width * height * channel
        kl.Input(shape=(224, 224, 3)),
        kl.Conv2D(filters=96, kernel_size=11, strides=4, activation="relu"),
        kl.MaxPool2D(pool_size=3, strides=2),
        kl.Conv2D(filters=256, kernel_size=5, padding="same", activation="relu"),
        kl.MaxPool2D(pool_size=3, strides=2),
        kl.Conv2D(filters=384, kernel_size=3, padding="same", activation="relu"),
        kl.Conv2D(filters=384, kernel_size=3, padding="same", activation="relu"),
        kl.Conv2D(filters=256, kernel_size=3, padding="same", activation="relu"),
        kl.MaxPool2D(pool_size=3, strides=2),

        # ubah jadi 1d, fully connected layer
        kl.Flatten(),
        kl.Dense(4096, activation="relu"),
        kl.Dropout(0.5),
        kl.Dense(4096, activation="relu"),
        kl.Dropout(0.5),
        # pake softmax agar memberikan probability score
        kl.Dense(num_of_classes, activation="softmax") # output
    ])

    return model

model = alexnet(num_classes)

In [29]:
model.compile(loss=BinaryCrossentropy(),
              optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [30]:
#Pre process Valid Dataset
def preprocess_valid_dataset():
    df = pd.read_csv(os.path.join(valid_csv_path, "_classes.csv"))
    df.columns = df.columns.str.strip().str.lower()

    images = []
    labels = []

    error_count = 0

    for index, row in df.iterrows():
        try:    
            image_path = os.path.join(valid_csv_path, row['filename'])
            image = Image.open(image_path)
            processed_image = preprocess_image(image)
            images.append(processed_image)

            label = [category for category in categories if row[category.lower()] == 1]
            labels.append(label)
        except Exception as e:
            print(f"Error {row['filename']} : {e}")
            error_count += 1
            
    print(error_count)
    return np.array(images), labels

    

In [31]:
#Generate Dataset
#Train data = converted_train_data
images_valid, labels_valid = preprocess_valid_dataset()

Error image_116-84-_jpg.rf.6da1d10839bfa6a693826a82dc6b1266.jpg : [Errno 2] No such file or directory: './Dataset/valid/image_116-84-_jpg.rf.6da1d10839bfa6a693826a82dc6b1266.jpg'
Error image_155_jpg.rf.74f4a0b24f4d873b7853d05e46ce4a4f.jpg : [Errno 2] No such file or directory: './Dataset/valid/image_155_jpg.rf.74f4a0b24f4d873b7853d05e46ce4a4f.jpg'
Error image_116-2023-07-12T173108-006_jpg.rf.74d560f02543f72c9bb16feba3dc6dc4.jpg : [Errno 2] No such file or directory: './Dataset/valid/image_116-2023-07-12T173108-006_jpg.rf.74d560f02543f72c9bb16feba3dc6dc4.jpg'
3


In [32]:
#Flatten and filter
flattened_labels_valid = [label[0] for label in labels_valid if label]
lookup_valid= tf.keras.layers.StringLookup(output_mode='int')
lookup_valid.adapt(flattened_labels_valid)

integer_labels_valid = lookup(flattened_labels_valid)
one_hot_labels_valid = tf.one_hot(integer_labels_valid, depth=6)

#Image
filtered_images_valid = [image for image, label in zip(images_valid, labels_valid) if label]

In [33]:
print(f"labels {len(integer_labels_valid)}, images {len(filtered_images_valid)}")

labels 1779, images 1779


In [34]:
converted_valid__data = tf.data.Dataset.from_tensor_slices((filtered_images_valid, one_hot_labels_valid)).batch(16)

In [35]:
# converted_valid__data = converted_valid__data.map(fix_shape)

In [36]:
for batch in converted_valid__data.take(1):
    images_batch, labels_batch = batch
    print("Batch shape:", images_batch.shape)
    print("Single image shape:", images_batch[0].shape)
    print("Batch labels shape:", labels_batch.shape)

Batch shape: (16, 224, 224, 3)
Single image shape: (224, 224, 3)
Batch labels shape: (16, 6)


In [37]:
#Compile
model.compile(loss=BinaryCrossentropy(),
              optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

In [38]:
#set Earlystooping
es = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)

In [39]:
lrate = ReduceLROnPlateau(monitor='val_loss', factor=0.15, patience=2, min_lr=0.00001, verbose=1)

In [40]:
#Fit
model.fit(converted_train__data, epochs=25, validation_data=converted_valid__data,
          callbacks=[lrate])

Epoch 1/25
384/384 [==============================] - 177s 457ms/step - loss: 0.3874 - accuracy: 0.1658 - val_loss: 0.3805 - val_accuracy: 0.1743 - lr: 1.0000e-04
Epoch 2/25
384/384 [==============================] - 175s 454ms/step - loss: 0.3740 - accuracy: 0.2239 - val_loss: 0.3649 - val_accuracy: 0.2485 - lr: 1.0000e-04
Epoch 3/25
384/384 [==============================] - 176s 456ms/step - loss: 0.3646 - accuracy: 0.2661 - val_loss: 0.3542 - val_accuracy: 0.2974 - lr: 1.0000e-04
Epoch 4/25
384/384 [==============================] - 178s 463ms/step - loss: 0.3559 - accuracy: 0.2915 - val_loss: 0.3534 - val_accuracy: 0.2996 - lr: 1.0000e-04
Epoch 5/25
384/384 [==============================] - 175s 455ms/step - loss: 0.3451 - accuracy: 0.3145 - val_loss: 0.3376 - val_accuracy: 0.3485 - lr: 1.0000e-04
Epoch 6/25
384/384 [==============================] - 175s 456ms/step - loss: 0.3320 - accuracy: 0.3467 - val_loss: 0.3253 - val_accuracy: 0.3783 - lr: 1.0000e-04
Epoch 7/25
384/384 [==

In [41]:
#Save Model
model.save('geoguesser_model.h5')

/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [42]:
#Load Model
loaded_model = tf.keras.models.load_model('geoguesser_model.h5')

In [43]:
#Compile
loaded_model.compile(loss=BinaryCrossentropy(),
              optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

In [44]:
# Continue training the loaded model
loaded_model.fit(converted_train__data, epochs=5, validation_data=converted_valid__data, callbacks=[es])


Epoch 1/5
384/384 [==============================] - 172s 446ms/step - loss: 0.1440 - accuracy: 0.6821 - val_loss: 0.4104 - val_accuracy: 0.4216
Epoch 2/5
384/384 [==============================] - 171s 444ms/step - loss: 0.1229 - accuracy: 0.7148 - val_loss: 0.4371 - val_accuracy: 0.4177
Epoch 3/5
384/384 [==============================] - 170s 443ms/step - loss: 0.1075 - accuracy: 0.7362 - val_loss: 0.4804 - val_accuracy: 0.4154
Epoch 4/5
384/384 [==============================] - 170s 443ms/step - loss: 0.0996 - accuracy: 0.7463 - val_loss: 0.5385 - val_accuracy: 0.4132
Epoch 5/5
384/384 [==============================] - 177s 459ms/step - loss: 0.0799 - accuracy: 0.7707 - val_loss: 0.5759 - val_accuracy: 0.4199


In [45]:
#Save it again#Save Model
loaded_model.save('geoguesser_model.h5')

In [46]:
#Load Model
loaded_model = tf.keras.models.load_model('geoguesser_model.h5')

In [47]:
#Compile
loaded_model.compile(loss=BinaryCrossentropy(),
              optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

In [48]:
# Continue training the loaded model
loaded_model.fit(converted_train__data, epochs=5, validation_data=converted_valid__data, callbacks=[es])

Epoch 1/5
384/384 [==============================] - 174s 452ms/step - loss: 0.0752 - accuracy: 0.7741 - val_loss: 0.5637 - val_accuracy: 0.4188
Epoch 2/5
384/384 [==============================] - 171s 446ms/step - loss: 0.0628 - accuracy: 0.7894 - val_loss: 0.5858 - val_accuracy: 0.4233
Epoch 3/5
384/384 [==============================] - 172s 447ms/step - loss: 0.0526 - accuracy: 0.7982 - val_loss: 0.6309 - val_accuracy: 0.4120
Epoch 4/5
384/384 [==============================] - 176s 458ms/step - loss: 0.0468 - accuracy: 0.8054 - val_loss: 0.6466 - val_accuracy: 0.4075
Epoch 5/5
384/384 [==============================] - 170s 443ms/step - loss: 0.0444 - accuracy: 0.8103 - val_loss: 0.6992 - val_accuracy: 0.4238


In [49]:
#Save model nya buat dipake 
loaded_model.save('geoguesser_model.h5')

In [50]:
# #Load Model
# loaded_model = tf.keras.models.load_model('geoguesser_modela.keras')
# #Compile
# loaded_model.compile(loss='categorical_crossentropy',
#               optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])
# # Continue training the loaded model
# loaded_model.fit(converted_train__data, epochs=5, validation_data=converted_valid__data, callbacks=[es])

In [51]:
#Geo Guesser func to predict
def guess_location(model_path, img_path):
    model = keras.models.load_model(model_path)
    
    #Pre process img_path 
    image = Image.open(img_path)
    img_array = preprocess_image(image)

    #Predict
    predictions = model.predict(np.expand_dims(img_array, axis=0))
    predicted_index = np.argmax(predictions, axis=1)[0]
    
    #Get the category name
    predicted_name = categories[predicted_index]
    return predicted_name

In [52]:
#Cara panggil func guess_location
print(guess_location('./geoguesser_model.h5','./Dataset/test/60-182849999999995-25-06746_jpg.rf.82f8a8d3e928f40aed73bd2964d52277.jpg'))

1/1 [==============================] - 0s 288ms/step
Japan
